In [2]:
import os
import numpy as np
import pandas as pd
from scipy import signal as sig
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr

c:\Users\Alli Ajagbe\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
ddir = os.path.join('.','precipitation')
pr = xr.open_mfdataset(f'{ddir}/*.nc')
print(f'pr: {pr}')

pr: <xarray.Dataset>
Dimensions:  (time: 1428, lon: 144, lat: 72)
Coordinates:
  * time     (time) datetime64[ns] 1901-01-01 1901-02-01 ... 2019-12-01
  * lon      (lon) float32 -178.8 -176.2 -173.8 -171.2 ... 173.8 176.2 178.8
  * lat      (lat) float32 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
Data variables:
    pr       (time, lat, lon) float32 dask.array<chunksize=(12, 72, 144), meta=np.ndarray>
Attributes: (12/23)
    CDI:                        Climate Data Interface version 1.9.10 (https:...
    source:                     Run ID = 2004151855. Data generated from:pre....
    institution:                KNMI Climate Explorer and Data held at Britis...
    Conventions:                CF-1.0
    title:                      CRU TS4.04 Precipitation  averaged over  5x 5...
    cdi:                        Climate Data Interface version 1.9.8 (https:/...
    ...                         ...
    geospatial_lon_resolution:  2.5
    climexp_url:                https://climexp.knmi

In [4]:
print(f'Dimensions: {pr.dims}')

Dimensions: Frozen({'time': 1428, 'lon': 144, 'lat': 72})


In [5]:
print(f'Attributes: {pr.attrs}')

Attributes: {'CDI': 'Climate Data Interface version 1.9.10 (https://mpimet.mpg.de/cdi)', 'source': 'Run ID = 2004151855. Data generated from:pre.2004011744.dtb', 'institution': 'KNMI Climate Explorer and Data held at British Atmospheric Data Centre, RAL, UK.', 'Conventions': 'CF-1.0', 'title': 'CRU TS4.04 Precipitation  averaged over  5x 5 grid boxes', 'cdi': 'Climate Data Interface version 1.9.8 (https://mpimet.mpg.de/cdi)', 'references': 'Information on the data is available at http://badc.nerc.ac.uk/data/cru/', 'comment': 'Access to these data is available to any registered CEDA user.', 'contact': 'support@ceda.ac.uk', 'cdo': 'Climate Data Operators version 1.9.8 (https://mpimet.mpg.de/cdo)', 'geospatial_lat_min': -90.0, 'geospatial_lat_max': 90.0, 'geospatial_lat_units': 'degrees_north', 'geospatial_lon_min': -180.0, 'geospatial_lon_max': 180.0, 'geospatial_lon_units': 'degrees_east', 'geospatial_lat_resolution': 2.5, 'geospatial_lon_resolution': 2.5, 'climexp_url': 'https://climex

In [6]:
pr_location = pr.sel(lat=31.25, lon=76.25)
print(f'pr_location: {pr_location}')

pr_location: <xarray.Dataset>
Dimensions:  (time: 1428)
Coordinates:
  * time     (time) datetime64[ns] 1901-01-01 1901-02-01 ... 2019-12-01
    lon      float32 76.25
    lat      float32 31.25
Data variables:
    pr       (time) float32 dask.array<chunksize=(12,), meta=np.ndarray>
Attributes: (12/23)
    CDI:                        Climate Data Interface version 1.9.10 (https:...
    source:                     Run ID = 2004151855. Data generated from:pre....
    institution:                KNMI Climate Explorer and Data held at Britis...
    Conventions:                CF-1.0
    title:                      CRU TS4.04 Precipitation  averaged over  5x 5...
    cdi:                        Climate Data Interface version 1.9.8 (https:/...
    ...                         ...
    geospatial_lon_resolution:  2.5
    climexp_url:                https://climexp.knmi.nl/select.cgi?cru4_pre
    time_coverage_start:        1901-01-01 00:00:00
    time_coverage_end:          2019-12-01 00:00:00


In [7]:
ddir = os.path.join('.','temperature')
tasmax = xr.open_mfdataset(f'{ddir}/*.nc')

In [9]:
pr.to_array()

,Array,Chunk
Bytes,56.48 MiB,486.00 kiB
Shape,"(1, 1428, 72, 144)","(1, 12, 72, 144)"
Count,476 Tasks,119 Chunks
Type,float32,numpy.ndarray


In [8]:
tasmax.to_array()

,Array,Chunk
Bytes,56.48 MiB,486.00 kiB
Shape,"(1, 1428, 72, 144)","(1, 12, 72, 144)"
Count,476 Tasks,119 Chunks
Type,float32,numpy.ndarray


In [11]:
tasmax

<xarray.Dataset>
Dimensions:  (time: 1428, lon: 144, lat: 72)
Coordinates:
  * time     (time) datetime64[ns] 1901-01-01 1901-02-01 ... 2019-12-01
  * lon      (lon) float32 -178.8 -176.2 -173.8 -171.2 ... 173.8 176.2 178.8
  * lat      (lat) float32 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<chunksize=(12, 72, 144), meta=np.ndarray>
Attributes: (12/23)
    CDI:                        Climate Data Interface version 1.9.10 (https:...
    source:                     Run ID = 2004151855. Data generated from:TMX ...
    institution:                KNMI Climate Explorer and Data held at Britis...
    Conventions:                CF-1.0
    title:                      CRU TS4.04 Maximum Temperature  averaged over...
    cdi:                        Climate Data Interface version 1.9.8 (https:/...
    ...                         ...
    geospatial_lon_resolution:  2.5
    climexp_url:                https://climexp.knmi.nl/select.cgi?cru4_tmx
    time_coverage_start:        1901-01-01 00:00:00
    time_coverage_end:          2019-12-01 00:00:00
    history:                    Fri Jun 18 08:25:04 2021: cdo -O -splityear /...
    CDO:                        Climate Data Operators version 1.9.10 (https:...